# Prepare Data

#### Data Sources

This notebook utilizes the following data sources:

1. **Worldscope Fundamentals**  
   Source: [LSEG - Worldscope Fundamentals](https://www.lseg.com/en/data-analytics/financial-data/company-data/fundamentals-data/worldscope-fundamentals)  
   Files should be named:  
     - `worldscope_a.csv`  
     - `worldscope_b.csv`  
   > _Note: The Worldscope dataset is divided into two subsets. The letter "a" in the column name denotes the main part of the dataset, containing company fundamentals as originally filed. The letter "b" denotes restatements. In case of restatements, we use the most recently avilable data following the logic specified in the code below._

2. **Refinitiv ESG Data**  
   Source: [LSEG - Refinitiv ESG Scores](https://www.lseg.com/en/data-analytics/sustainable-finance/esg-scores)  
   Files should be named:  
     - `refinitiv_esg.csv`

3. **MSCI ESG Ratings**  
   Source: [MSCI ESG Ratings](https://www.msci.com/data-and-analytics/sustainability-solutions/esg-ratings)  
   Files should be named:  
     - `ESG Ratings Timeseries Expanded {i}.csv`  
       _(where `{i}` corresponds to the year of the dataset)_
   > _Note: The MSCI dataset is split into several datasets, each containing rating data for a specific year. We concatenate the datasets following the logic specified in the code below._

#### Imports

In [ ]:
import pandas as pd
import os
import glob

### 1. Create Company-Level Data Frame

In [ ]:
# Load companies and indices from SRN API
# This is our sample
companies = pd.read_json("https://api.sustainabilityreportingnavigator.com/api/companies")
indices = pd.read_json("https://api.sustainabilityreportingnavigator.com/api/indices")

companies['EuroStoxx600'] = ['1cc738c1-e6b1-4f2b-8bec-2d963118de59' in x for x in companies['indices']]
companies_se600 = companies[companies['EuroStoxx600'] == True]
companies_se600

In [ ]:
# Create company data frame
firm_data = companies_se600[['id', 'name', 'isin', 'country', 'sector']]

# Create rump times series data frame
years = list(range(2014, 2024))
firm_data = firm_data.loc[firm_data.index.repeat(len(years))]
firm_data['year'] = years * (len(firm_data) // len(years))
firm_data.reset_index(drop=True, inplace=True)

firm_data

### 2. Merge with Worldscope Data

In [ ]:
# Define relevant Worldscope variables
ws_vars =  {
    # Identifiying
    'ITEM6008': 'isin',
    'year_':    'year',
    'ITEM6011': 'industry_group',
    'ITEM5601': 'worldscope_ticker',
    'ITEM6105': 'worldscope_permno',
    # P&L
    'ITEM1001': 'sales',
    'ITEM7240': 'salesUSD',
    'ITEM1148': 'deprec',
    'ITEM1151': 'deprec_amort',
    'ITEM18274':'impairment_ppe',
    'ITEM18225':'impairment_gw',
    'ITEM1250': 'opinc',
    'ITEM1751': 'netinc',
    'ITEM7250': 'netincUSD',
    # B/S
    'ITEM2300': 'assets_asreported',
    'ITEM2999': 'assets',
    'ITEM2301': 'ppe_gross',
    'ITEM2401': 'deprec_accum',
    'ITEM2501': 'ppe_net',
    'ITEM7230': 'assetsUSD',
    'ITEM3255': 'debt', 
    'ITEM3260': 'risk_provisions', 
    'ITEM1302': 'op_provisions',
    'ITEM7210': 'mkcapUSD',
    'ITEM7220': 'equityUSD',
}

keep_cols = list(ws_vars.keys()) + ['freq']   # Need freq for restatements

In [ ]:
# 1. Load and clean Worldscope data (retrieved in April 2025 for universe)
# A and B files denote original and restated data
ws_raw = (
    pd.concat([
        pd.read_csv(r'..\data\raw\datasets\worldscope_a.csv',
                    usecols=lambda c: c in keep_cols),
        pd.read_csv(r'..\data\raw\datasets\worldscope_b.csv',
                    usecols=lambda c: c in keep_cols)
    ], ignore_index=True)
    .sort_values(['ITEM6008', 'year_', 'freq'], ascending=[True, True, False])
)

# Keep the *latest* restatement and fill remaining gaps
first_in_group = ws_raw.groupby(['ITEM6008', 'year_']).cumcount() == 0
ws_filled      = (
    ws_raw.groupby(['ITEM6008', 'year_'])
          .transform('ffill')
          .fillna(ws_raw.groupby(['ITEM6008', 'year_']).transform('bfill'))
)
ws_clean = (ws_raw.loc[first_in_group]
                     .fillna(ws_filled)
                     .reset_index(drop=True))

In [ ]:
# 2. Rename and keep only relevant columns
ws_clean = ws_clean.rename(columns=ws_vars)[ws_vars.values()]

In [ ]:
# 3. Build a lagged marketcapUSD variable
ws_lagged = (
    ws_clean[['isin', 'year', 'mkcapUSD']]
      .assign(year = lambda d: d['year'] + 1,
              mkcapUSD_lagged = lambda d: d.pop('mkcapUSD'))
)

In [ ]:
# 4. Merge with firm data
firm_data = (
    firm_data
    .merge(ws_clean,  on=['isin', 'year'], how='left')   # current year vars
    .merge(ws_lagged, on=['isin', 'year'], how='left')   # lagged mkcap
)

### 3. Merge with Refinitiv Data

In [ ]:
# Load the data from Refinitiv
ref = pd.read_csv(r'..\data\raw\datasets\refinitiv_esg.csv')
ref

In [ ]:
# Define relevant Refinitiv variables
ref_vars = {
    'ESGScore': 'ref_esg_score',
    'ESGCombinedScore': 'ref_esg_combined_score',
    'ESGCControversiesScore': 'ref_esg_controversies_score',
    'EnvironmentPillarScore': 'ref_env_score',
    'SocialPillarScore': 'ref_soc_score',
    'GovernancePillarScore': 'ref_gov_score',
    'ESGResourceUseScore': 'ref_res_use_score',
    'ESGEmissionsScore': 'ref_emissions_score',
    'ESGInnovationScore': 'ref_innovation_score',
    'ESGWorkforceScore': 'ref_workforce_score',
    'ESGHumanRightsScore': 'ref_human_rights_score',
    'ESGCommunityScore': 'ref_community_score',
    'ESGProductResponsibilityScore': 'ref_product_responsibility_score',
    'ESGManagementScore': 'ref_management_score',
    'ESGShareholdersScore': 'ref_shareholders_score',
    'ESGCsrStrategyScore': 'ref_csr_strategy_score',
    'CO2EquivalentsEmissionDirectScope1': 'ref_scope1',
    'CO2EquivalentsEmissionIndirectScope2': 'ref_scope2',
    'CO2EquivalentsEmissionIndirectScope3': 'ref_scope3',
    'CO2EquivalentsEmissionTotal': 'ref_total',
    'ClimateChangeCommercialRisksOpportunities': 'climate_commercial_riskopp_aware',
    'CarbonOffsetsCredits': 'carbon_offsets',
    'EmissionReductionTargetPercentage': 'emission_reduction_target',
    'EmissionReductionTargetYear': 'emission_reduction_target_year',
    'EmissionsTrading': 'emission_trading',
    'CsrSustainabilityCommittee': 'csr_sust_committee',
    'CsrSustainabilityExternalAudit': 'csr_assurance',
    'CsrSustainabilityExternalAuditorName': 'csr_assurance_name',
    'CsrSustainabilityReportGlobalActivities': 'csr_global_scope',
    'CsrSustainabilityReporting': 'csr_separate_report',
    'ESGReportingScope': 'esg_reporting_scope',
    'GlobalCompactSignatory': 'global_compact_signatory',
    'GriReportGuidelines': 'gri_reporting',
    'IntegratedStrategyInMdAndA': 'integrated_mda',
    'SustainabilityCompensationIncentives': 'esg_compensation', 
}

In [ ]:
# Function to merge firm data with Refinitiv data
def merge_with_ref(firm_data, ref, ref_vars):

    # Reshape df
    ref_pivot = ref.pivot(index=['isin', 'year'], columns='fieldname', values='value').reset_index()

    # Rename columns
    ref_pivot = ref_pivot.rename(columns=ref_vars)
    ref_pivot = ref_pivot.loc[:, ['isin', 'year'] + list(ref_vars.values())]

    # Merge
    firm_data = pd.merge(firm_data, ref_pivot, on=['isin', 'year'], how='left')

    # Prepare lagged data
    ref_lagged = ref_pivot[['isin', 'year', 'ref_esg_score', 'ref_esg_controversies_score']].copy()
    ref_lagged['year'] += 1

    # Rename lagged columns
    ref_lagged = ref_lagged.rename(columns={
        'ref_esg_score': 'ref_esg_score_lagged',
        'ref_esg_controversies_score': 'ref_esg_controversies_score_lagged'
    })

    # Merge lagged data
    firm_data = pd.merge(firm_data, ref_lagged, on=['isin', 'year'], how='left')

    return firm_data

In [ ]:
firm_data = merge_with_ref(firm_data, ref, ref_vars)
firm_data

### 4. Merge with MSCI Data

In [ ]:
# Configuration
dir = r'..\data\raw\datasets\msci'
start = 2007
end = 2023
vars = {
    'ISSUER_NAME': 'name',
    'ISSUER_ISIN': 'isin',
    'AS_OF_DATE': 'msci_as_of_date',
    'IVA_INDUSTRY': 'msci_industry',
    'IVA_RATING_DATE': 'msci_rating_date',
    'IVA_COMPANY_RATING': 'msci_company_rating',
    'ENVIRONMENTAL_PILLAR_SCORE': 'msci_env_score',
    'SOCIAL_PILLAR_SCORE': 'msci_soc_score',
    'GOVERNANCE_PILLAR_SCORE': 'msci_gov_score'
}

In [ ]:
# Function to load MSCI data
# MSCI data is provided in multiple CSV files, one for each year, with the exception of 2007-2012, which are combined in a single file
def load_msci_data(dir, start, end, vars):

    data_frames = []

    for i in range(start, end):
        if i in range(2007, 2013) and i != start:
            continue

        file_name = (
            'ESG Ratings Timeseries Expanded 2007 to 2012.csv'
            if i in range(2007, 2013)
            else f'ESG Ratings Timeseries Expanded {i}.csv'
        )
    
        read = pd.read_csv(os.path.join(dir, file_name), usecols=list(vars.keys()), sep=';').rename(columns=vars)
        data_frames.append(read)

    msci = pd.concat(data_frames, ignore_index=True)

    # Build yearly data and use the last rating of the year
    msci['msci_as_of_date'] = pd.to_datetime(msci['msci_as_of_date'], format='%Y%m%d')
    msci['year'] = msci['msci_as_of_date'].dt.year
    msci = msci.sort_values(by=['isin', 'msci_as_of_date'], ascending=[True, True])
    msci = msci.groupby(['isin', 'year']).last().reset_index()

    return msci


In [ ]:
msci = load_msci_data(dir, start, end, vars)
msci

In [ ]:
# Create lagged MSCI data
# Copy the MSCI data
msci_lagged = msci.copy()

# Create lagged year
msci_lagged['lagged_year'] = msci_lagged['year'] + 1  # This will match the firm_data year

# Rename columns
msci_lagged = msci_lagged.rename(columns={
    'msci_as_of_date': 'msci_as_of_date_lagged',
    'msci_rating_date': 'msci_rating_date_lagged',
    'msci_company_rating': 'msci_company_rating_lagged',
    'msci_env_score': 'msci_env_score_lagged',
    'msci_soc_score': 'msci_soc_score_lagged',
    'msci_gov_score': 'msci_gov_score_lagged'
})

In [ ]:
# Function to merge firm data with MSCI data
def merge_with_msci(firm_data, msci):
    msci = msci.drop(columns=['name'])
    firm_data = pd.merge(firm_data, msci, on=['isin', 'year'], how='left')

    # Merge with lagged MSCI data
    firm_data = pd.merge(
        firm_data,
        msci_lagged[
            [
                'isin',
                'lagged_year',
                'msci_as_of_date_lagged',
                'msci_rating_date_lagged',
                'msci_company_rating_lagged',
                'msci_env_score_lagged',
                'msci_soc_score_lagged',
                'msci_gov_score_lagged'
            ]
        ],
        left_on=['isin', 'year'],
        right_on=['isin', 'lagged_year'],
        how='left'
    ).drop(columns=['lagged_year'])
    
    return firm_data

In [ ]:
firm_data = merge_with_msci(firm_data, msci)
firm_data

### 5. Save File

In [ ]:
firm_data.to_csv(r'..\data\raw\datasets\firm_data.csv', index=False)